In [1]:
import pandas as pd
import re
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree.export import export_text

Функция, которая получает начала строк, в которых имеются релевантные данные. sample - информация об испытуемых , series - гены.

In [2]:
def get_name_sample_series(txt):
    sample = []
    series = []
    i = 0
    try:
        with open(txt, 'r') as f:
            for line in f:
                l = line.split('\t')
                if ('!Sample' in l[0]):
                    sample.append(l[0])
                else:
                    if ('!series' in l[0] or i == 1):
                        i = 1
                        series.append(l[0])
    except IOError:
        print("An IOError has occurred!")
    finally:
        return sample, series[1:-1]

Преобразование входных данных в словарь

In [3]:
def dict_data(data):
    dict_data = {}
    for i in range(1,len(data)):
        dict_data[i-1] = re.sub(r"[\'\"\n]","",data[i])
    return dict_data

Получение DataFrame для sample, series из словарей

In [4]:
def get_df(txt):
    sample, series = get_name_sample_series(txt)
    try:
        with open(txt, 'r') as f:
            dict_sample = {}
            dict_series = {}
            for line in f:
                l = line.split('\t')
                if (l[0] in sample):
                    dict_sample[re.sub(r"!", "" ,l[0])] = dict_data(l)
                else:
                    if (l[0] in series):
                        dict_series[re.sub(r"\"", "", l[0])] = dict_data(l)
                        
    except IOError:
        print("An IOError has occurred!")
    finally:
        return pd.DataFrame(dict_sample), pd.DataFrame(dict_series)

Поиск дубликатов в данных по генам

In [5]:
def find_duplicate(df_series):
    duplicate = []
    dup = pd.DataFrame({'duplicate': df_series.T[1:].duplicated()})
    for i in range(dup.count()[0]):
        if dup.iloc[i].duplicate == True:
            duplicate.append(dup.iloc[i].name)
    return duplicate

Извлечение номеров испытуемых в выборках control, low_dose, high_dose

In [6]:
def get_control_low_high(df_sample):
    count_liver = df_sample.Sample_title.count()
    control = []
    low = []
    high = []
    for i in range(count_liver):
        if ('control' in df_sample.Sample_title[i]):
            control.append(i)
        if ('Low' in df_sample.Sample_title[i]):
            low.append(i)
        if ('High' in df_sample.Sample_title[i]):
            high.append(i)
    return control, low, high

Получение средних значений по каждому гену в выборках control, low_dose, high_dose

In [7]:
def get_series_mean(df_sample, df_series):
    control, low, high = get_control_low_high(df_sample)
    for col in df_series.columns[1:]:
        df_series[col] = pd.to_numeric(df_series[col])
    s_mean_control = df_series.loc[control].mean(axis=0)
    s_mean_low = df_series.loc[low].mean(axis=0)
    s_mean_high = df_series.loc[high].mean(axis=0)
    return s_mean_control, s_mean_low, s_mean_high

Построение дерева решений sample1 vs sample2

In [8]:
def s1_vs_s2 (sample1, sample2):
    clf = DecisionTreeClassifier(random_state=0, max_depth=2)
    X = pd.DataFrame({'s1': sample1, 's2':sample2})
    clf.fit(X.T, [0,1])
    r = export_text(clf)
    return r, X.iloc[int(r.split()[1].split('_')[1])]

In [9]:
df_sample, df_series = get_df('GSE26728_series_matrix.txt')

In [10]:
df_sample

,Sample_title,Sample_geo_accession,Sample_status,Sample_submission_date,Sample_last_update_date,Sample_type,Sample_channel_count,Sample_source_name_ch1,Sample_organism_ch1,Sample_characteristics_ch1,...,Sample_contact_name,Sample_contact_email,Sample_contact_laboratory,Sample_contact_institute,Sample_contact_address,Sample_contact_city,Sample_contact_zip/postal_code,Sample_contact_country,Sample_supplementary_file,Sample_data_row_count
0,liver_control_rep1,GSM658075,Public on Sep 21 2011,Jan 19 2011,Sep 23 2011,RNA,1,"liver, control, replicate 1",Mus musculus,treatment: control,...,"Pascal,GP,Martin","Pascal.Martin@inra.fr, pascmart@iu.edu",ToxAlim,INRA,"180 chemin de Tournefeuille, BP 93173",Toulouse Cedex 3,31027,France,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM658n...,22514
1,liver_control_rep2,GSM658076,Public on Sep 21 2011,Jan 19 2011,Sep 23 2011,RNA,1,"liver, control, replicate 2",Mus musculus,treatment: control,...,"Pascal,GP,Martin","Pascal.Martin@inra.fr, pascmart@iu.edu",ToxAlim,INRA,"180 chemin de Tournefeuille, BP 93173",Toulouse Cedex 3,31027,France,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM658n...,22514
2,liver_control_rep3,GSM658077,Public on Sep 21 2011,Jan 19 2011,Sep 23 2011,RNA,1,"liver, control, replicate 3",Mus musculus,treatment: control,...,"Pascal,GP,Martin","Pascal.Martin@inra.fr, pascmart@iu.edu",ToxAlim,INRA,"180 chemin de Tournefeuille, BP 93173",Toulouse Cedex 3,31027,France,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM658n...,22514
3,liver_control_rep4,GSM658078,Public on Sep 21 2011,Jan 19 2011,Sep 23 2011,RNA,1,"liver, control, replicate 4",Mus musculus,treatment: control,...,"Pascal,GP,Martin","Pascal.Martin@inra.fr, pascmart@iu.edu",ToxAlim,INRA,"180 chemin de Tournefeuille, BP 93173",Toulouse Cedex 3,31027,France,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM658n...,22514
4,liver_control_rep5,GSM658079,Public on Sep 21 2011,Jan 19 2011,Sep 23 2011,RNA,1,"liver, control, replicate 5",Mus musculus,treatment: control,...,"Pascal,GP,Martin","Pascal.Martin@inra.fr, pascmart@iu.edu",ToxAlim,INRA,"180 chemin de Tournefeuille, BP 93173",Toulouse Cedex 3,31027,France,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM658n...,22514
5,liver_control_rep6,GSM658080,Public on Sep 21 2011,Jan 19 2011,Sep 23 2011,RNA,1,"liver, control, replicate 6",Mus musculus,treatment: control,...,"Pascal,GP,Martin","Pascal.Martin@inra.fr, pascmart@iu.edu",ToxAlim,INRA,"180 chemin de Tournefeuille, BP 93173",Toulouse Cedex 3,31027,France,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM658n...,22514
6,liver_BPALowDose_rep1,GSM658081,Public on Sep 21 2011,Jan 19 2011,Sep 23 2011,RNA,1,"liver, exposed to BPA at 50 µg/Kg/day, replica...",Mus musculus,treatment: exposed to BPA at 50 µg/Kg/day,...,"Pascal,GP,Martin","Pascal.Martin@inra.fr, pascmart@iu.edu",ToxAlim,INRA,"180 chemin de Tournefeuille, BP 93173",Toulouse Cedex 3,31027,France,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM658n...,22514
7,liver_BPALowDose_rep2,GSM658082,Public on Sep 21 2011,Jan 19 2011,Sep 23 2011,RNA,1,"liver, exposed to BPA at 50 µg/Kg/day, replica...",Mus musculus,treatment: exposed to BPA at 50 µg/Kg/day,...,"Pascal,GP,Martin","Pascal.Martin@inra.fr, pascmart@iu.edu",ToxAlim,INRA,"180 chemin de Tournefeuille, BP 93173",Toulouse Cedex 3,31027,France,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM658n...,22514
8,liver_BPALowDose_rep3,GSM658083,Public on Sep 21 2011,Jan 19 2011,Sep 23 2011,RNA,1,"liver, exposed to BPA at 50 µg/Kg/day, replica...",Mus musculus,treatment: exposed to BPA at 50 µg/Kg/day,...,"Pascal,GP,Martin","Pascal.Martin@inra.fr, pascmart@iu.edu",ToxAlim,INRA,"180 chemin de Tournefeuille, BP 93173",Toulouse Cedex 3,31027,France,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM658n...,22514
9,liver_BPALowDose_rep4,GSM658084,Public on Sep 21 2011,Jan 19 2011,Sep 23 2011,RNA,1,"liver, exposed to BPA at 50 µg/Kg/day, replica...",Mus musculus,treatment: exposed to BPA at 50 µg/Kg/day,...,"Pascal,GP,Martin","Pascal.Martin@inra.fr, pascmart@iu.edu",ToxAlim,INRA,"180 chemin de

In [11]:
df_series

,ID_REF,A_51_P100034,A_51_P100063,A_51_P100099,A_51_P100155,A_51_P100174,A_51_P100181,A_51_P100227,A_51_P100246,A_51_P100289,...,A_52_P995381,A_52_P996032,A_52_P996473,A_52_P99665,A_52_P99670,A_52_P997209,A_52_P997449,A_52_P99807,A_52_P99848,A_52_P99888
0,GSM658075,12.630,5.828,7.638,10.947,6.632,6.655,8.205,9.787,8.455,...,7.651,8.821,5.901,7.842,7.712,7.734,10.070,8.176,6.951,7.445
1,GSM658076,11.899,6.598,7.635,10.582,6.089,6.788,8.194,9.590,8.218,...,7.246,8.740,6.098,8.468,7.174,8.105,9.784,7.628,7.187,6.946
2,GSM658077,12.393,5.984,7.216,10.406,6.012,6.584,8.335,9.846,8.639,...,7.240,8.799,6.142,8.023,7.613,7.841,9.009,7.993,7.154,7.340
3,GSM658078,12.539,5.986,7.767,11.054,6.190,6.822,8.146,9.733,8.550,...,7.352,8.063,6.114,7.959,7.846,7.631,10.071,7.717,6.452,7.277
4,GSM658079,12.368,6.089,7.516,10.770,6.260,6.742,8.153,9.027,8.273,...,7.582,8.553,5.856,8.037,8.000,7.893,9.762,6.355,7.582,7.379
5,GSM658080,12.361,6.163,7.378,10.402,6.131,6.782,8.252,9.852,8.413,...,6.988,8.428,5.963,7.943,7.264,8.232,9.535,6.668,6.585,7.173
6,GSM658081,12.332,6.384,7.171,10.610,5.993,6.805,8.209,9.363,8.399,...,6.974,7.464,6.103,8.083,7.693,7.570,9.876,6.699,7.758,7.800
7,GSM658082,12.148,6.410,7.558,10.512,6.274,6.822,8.303,9.166,8.254,...,7.362,8.489,6.444,8.218,6.954,8.146,9.999,7.256,6.601,7.264
8,GSM658083,12.032,5.984,7.571,10.492,6.002,6.734,8.249,9.492,8.398,...,7.042,8.437,5.413,8.278,7.569,7.550,10.084,6.803,6.838,7.402
9,GSM658084,11.955,6.158,7.607,10.485,6.011,6.954,8.120,9.592,8.457,...,7.047,8.243,6.283,7.079,8.165,7.820,10.064,8.208,6.869,7.138


Количество исследуемых особей

In [12]:
df_sample.Sample_title.count()

18

Количество исследуемых генов

In [13]:
len(df_series.columns[1:])

22514

Поиск дубликатов в данных по генам

In [14]:
duplicate = find_duplicate(df_series)

Количество дубликатов

In [15]:
len(duplicate)

30

Гены дубликаты и их значения

In [16]:
df_series[duplicate]

,A_51_P120295,A_51_P128973,A_51_P145785,A_51_P160713,A_51_P164504,A_51_P171999,A_51_P182631,A_51_P235945,A_51_P236303,A_51_P287810,...,A_51_P428483,A_51_P461703,A_51_P474701,A_51_P486046,A_52_P232508,A_52_P319093,A_52_P412506,A_52_P436206,A_52_P527944,A_52_P684378
0,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,...,16.000,16.000,16.000,16.000,15.994,16.000,16.000,16.000,16.000,16.000
1,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,...,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994
2,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,...,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000
3,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,...,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000
4,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,...,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000
5,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,...,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000,16.000
6,15.984,15.984,15.984,15.984,15.984,15.984,15.984,15.984,15.984,15.984,...,15.984,15.984,15.984,15.984,15.984,15.984,15.984,15.984,15.984,15.984
7,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,...,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994
8,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,...,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994,15.994
9,15.996,15.996,15.996,15.996,15.996,15.996,15.996,15.996,15.996,15.996,...,15.996,15.996,15.996,15.996,15.996,15.996,15.996,15.996,15.996,15.996


Получение усредненных значений по каждой из выборок control, low_dose, high_dose (без удаления дубликатов)

In [17]:
s_mean_control, s_mean_low, s_mean_high = get_series_mean(df_sample, df_series)

In [18]:
s_mean_control

A_51_P100034    12.365000
A_51_P100063     6.108000
A_51_P100099     7.525000
A_51_P100155    10.693500
A_51_P100174     6.219000
                  ...    
A_52_P997209     7.906000
A_52_P997449     9.705167
A_52_P99807      7.422833
A_52_P99848      6.985167
A_52_P99888      7.260000
Length: 22514, dtype: float64

In [19]:
s_mean_low

A_51_P100034    12.148500
A_51_P100063     6.133333
A_51_P100099     7.509667
A_51_P100155    10.578833
A_51_P100174     6.077667
                  ...    
A_52_P997209     7.734833
A_52_P997449    10.062500
A_52_P99807      7.397167
A_52_P99848      7.095833
A_52_P99888      7.335833
Length: 22514, dtype: float64

In [20]:
s_mean_high

A_51_P100034    12.182333
A_51_P100063     6.648000
A_51_P100099     7.486667
A_51_P100155    10.473667
A_51_P100174     6.199500
                  ...    
A_52_P997209     7.924167
A_52_P997449     9.782667
A_52_P99807      6.939167
A_52_P99848      7.304333
A_52_P99888      7.363500
Length: 22514, dtype: float64

Дерево решений для классификации control vs low_dose, ген разбиения

In [21]:
r1, control_vs_low = s1_vs_s2(s_mean_control,s_mean_low)

In [22]:
print(r1)
print(control_vs_low)

|--- feature_15347 <= 13.86
|   |--- class: 0
|--- feature_15347 >  13.86
|   |--- class: 1

s1    13.796333
s2    13.924500
Name: A_52_P315030, dtype: float64


Дерево решений для классификации control vs high_dose, ген разбиения

In [23]:
r2, control_vs_high = s1_vs_s2(s_mean_control,s_mean_high)

In [24]:
print(r2)
print(control_vs_high)

|--- feature_15347 <= 13.83
|   |--- class: 0
|--- feature_15347 >  13.83
|   |--- class: 1

s1    13.796333
s2    13.872333
Name: A_52_P315030, dtype: float64
